In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

#predictive modelling
import xgboost

In [ ]:
#loading data. 
#reminder: NEED TO IMPORT THE CORRECT DATASET, DOUBLE CHECK
#reminder: handle the features and cleaning then touch on visualisation

seattle = pd.read_csv()

In [ ]:
seattle.head() 